In [1]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np
import keras2onnx
import onnxruntime
print(f"Tensoflow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
from models import endpoint_in_model
from losses import euclidean_distance_loss,endpoint_loss
from data_processing.data_loader import dataset_loader
from config import params

from time import time
import keract



os.environ['CUDA_VISIBLE_DEVICES']="1"
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Tensoflow Version: 2.3.0
Keras Version: 2.4.0
Num GPUs Available:  1


In [2]:
"""
dataset_dir = '/netpool/work/gpu-3/users/malyalasa/New_folder/tf_records'
ds_loader = dataset_loader(tfrec_dir=dataset_dir, batch_size=32)
ds_train, ds_valid, ds_test = ds_loader.build_dataset()
"""
dataset_dir = '/netpool/work/gpu-3/users/malyalasa/New_folder/tf_records'
ds_loader = dataset_loader(tfrec_dir=dataset_dir, batch_size=32)
#ds_train, ds_valid, ds_test = ds_loader.build_dataset()
ds_train, ds_valid, ds_test = ds_loader.build_scenario_dataset(consider_scenes=8,no_train_scene=6,no_valid_scene=1,no_test_scene=1)


Building Dataset.......

Number of records, Train files:464, validation files:96, Test Files:23
((TensorSpec(shape=(None, 1536, 1536), dtype=tf.int8, name=None),
  TensorSpec(shape=(None, None), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 25, 2), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 25, 2), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 25, 2), dtype=tf.float32, name=None),
  TensorSpec(shape=(None,), dtype=tf.string, name=None)),
 TensorSpec(shape=(None, 25, 2), dtype=tf.float32, name=None))


In [3]:
# load model wwith CustomMaskLayer
pp_model = tf.keras.models.load_model('/netpool/work/gpu-3/users/malyalasa/New_folder/wandb/latest-run/files/model-best.h5',compile=False,custom_objects={'CustomMaskLayer':endpoint_in_model.CustomMaskLayer})

#pp_model = tf.keras.models.load_model('/netpool/work/gpu-3/users/malyalasa/New_folder/wandb/run-20210709_111150-1dpjz991/files/model-best.h5',compile=False,custom_objects={'CustomMaskLayer':endpoint_in_model.CustomMaskLayer})


pp_model.summary()
# Compile the model
opt = tf.keras.optimizers.Adam(learning_rate=0.02)
pp_model.compile(
        optimizer=opt, 
        loss=[euclidean_distance_loss,endpoint_loss],
        loss_weights=params.get("loss_weights"), metrics=params.get("metric"))

# evaluate
test_loss,test_accuracy = pp_model.evaluate(ds_test)


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1536, 1536,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 767, 767, 16) 160         input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 767, 767, 16) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 767, 767, 16) 64          leaky_re_lu[0][0]                
_______________________________________________________________________________________

In [4]:
data = np.load('/netpool/work/gpu-3/users/malyalasa/New_folder/rosbag2numpy/dataset/sample_gridmap.npz')
input_sample = [data[f'{file}'] for file in data.files]

#print(len(input_sample))
pp_model.predict(input_sample)

array([[[ 352.27362, -666.91986],
        [ 336.72354, -595.63116],
        [ 330.86682, -522.6401 ],
        [ 348.06293, -449.03418],
        [ 381.0513 , -385.19705],
        [ 401.53854, -361.48187],
        [ 404.96503, -390.10413],
        [ 402.79446, -428.60345],
        [ 396.1076 , -466.89886],
        [ 388.77612, -508.43182],
        [ 381.21564, -548.4617 ],
        [ 372.00912, -591.8673 ],
        [ 359.3354 , -632.5618 ],
        [ 352.51318, -652.2194 ],
        [ 350.25333, -659.9084 ],
        [ 348.1363 , -667.8056 ],
        [ 346.98016, -673.402  ],
        [ 346.9462 , -675.5555 ],
        [ 347.44965, -676.85046],
        [ 347.76608, -676.5613 ],
        [ 347.5745 , -677.4424 ],
        [ 347.5085 , -677.4831 ],
        [ 347.5333 , -677.4007 ],
        [ 347.5713 , -677.2248 ],
        [ 347.67737, -677.6657 ]]], dtype=float32)

In [5]:
activations = keract.get_activations(pp_model, input_sample, auto_compile=True)

In [6]:
activations_dir = '/netpool/work/gpu-3/users/malyalasa/New_folder/rosbag2numpy/logging/after_norm_acts1'

keract.display_activations(activations, cmap=None, save=True, directory=activations_dir, data_format='channels_last', fig_size=(10, 10), reshape_1d_layers=False)


input_1 (1, 1536, 1536, 1) 
Grid_origin_res (1, 3) 
Left_boundary (1, 25, 2) 
Right_boundary (1, 25, 2) 
Car_loc (1, 3) 
Initial_path (1, 25, 2) 
conv2d (1, 767, 767, 16) 
leaky_re_lu (1, 767, 767, 16) 
batch_normalization (1, 767, 767, 16) 
average_pooling2d (1, 191, 191, 16) 
conv2d_1 (1, 189, 189, 32) 
leaky_re_lu_1 (1, 189, 189, 32) 
batch_normalization_1 (1, 189, 189, 32) 
conv2d_2 (1, 189, 189, 8) 
conv2d_3 (1, 189, 189, 1) 
flatten (1, 35721) 
reshape (1, 50) 
reshape_1 (1, 50) 
reshape_2 (1, 50) 
concatenate (1, 6) 
concatenate_1 (1, 35877) 
dense (1, 16) 
batch_normalization_2 (1, 16) 
dropout (1, 16) 
dense_1 (1, 50) 
custom_mask_layer (1, 25, 2) 
dropout_1 (1, 50) 
reshape_3 (1, 50) 
add (1, 50) 
dense_2 (1, 50) 
reshape_4 (1, 25, 2) 
